In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
data = pd.read_csv('/content/Data.csv')
data.head()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...,NaN,NaN
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.,NaN,NaN
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64636 entries, 0 to 64635
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            64636 non-null  int64 
 1   Situation             64636 non-null  object
 2   emotion               64632 non-null  object
 3   empathetic_dialogues  64636 non-null  object
 4   labels                64636 non-null  object
 5   Unnamed: 5            113 non-null    object
 6   Unnamed: 6            5 non-null      object
dtypes: int64(1), object(6)
memory usage: 3.5+ MB


In [ ]:
data = data.drop('Unnamed: 0',axis=1)
data = data.drop('Situation',axis=1)
data = data.drop('emotion',axis=1)
data = data.drop('Unnamed: 5',axis=1)
data = data.drop('Unnamed: 6',axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64636 entries, 0 to 64635
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   empathetic_dialogues  64636 non-null  object
 1   labels                64636 non-null  object
dtypes: object(2)
memory usage: 1010.1+ KB


In [ ]:
import re

def clean_conversation(text):
    if isinstance(text, str):
        text = re.sub(r'^Customer\s*:\s*', '', text)
        text = re.sub(r'\s*\nAgent\s*:\s*$', '', text)
        return text.strip()
    return text

data_input = data['empathetic_dialogues'].astype(str)
data_input = data_input.apply(clean_conversation)
data_input[:4]

,empathetic_dialogues
0,I remember going to see the fireworks with my ...
1,This was a best friend. I miss her.
2,We no longer talk.
3,"Was this a friend you were in love with, or ju..."


In [ ]:
input_data = [str(item) for item in data_input]
input_data[:4]

['I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.',
 'This was a best friend. I miss her.',
 'We no longer talk.',
 'Was this a friend you were in love with, or just a best friend?']

In [ ]:
output_data = [str(item) for item in data['labels']]
output_data[:4]

['Was this a friend you were in love with, or just a best friend?',
 'Where has she gone?',
 'Oh was this something that happened because of an argument?',
 'This was a best friend. I miss her.']

In [ ]:
input_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
output_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
ip_tokens = input_tokenizer(input_data,padding=True,truncation=True,return_tensors='pt')
input_tokens = ip_tokens['input_ids']
input_attention_mask = ip_tokens['attention_mask']
display(input_tokens[2])
display(input_attention_mask[2])

tensor([ 101, 2057, 2053, 2936, 2831, 1012,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0])

tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [ ]:
op_tokens = output_tokenizer(output_data,padding=True,truncation=True,return_tensors='pt')
output_tokens = op_tokens['input_ids']
output_attention_mask = op_tokens['attention_mask']
display(output_tokens[2])
display(output_attention_mask[2])

tensor([ 101, 2821, 2001, 2023, 2242, 2008, 3047, 2138, 1997, 2019, 6685, 1029,
         102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [ ]:
class Seq2SeqDataset(Dataset):
    def __init__(self, input_tokens, input_mask, output_tokens, output_mask):
        self.input_tokens = input_tokens
        self.input_mask = input_mask
        self.output_tokens = output_tokens
        self.output_mask = output_mask

    def __len__(self):
        return len(self.input_tokens)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_tokens[idx],
            "attention_mask": self.input_mask[idx],
            "labels": self.output_tokens[idx]
        }

dataset = Seq2SeqDataset(input_tokens, input_attention_mask, output_tokens, output_attention_mask)

In [ ]:
batch_size = 8
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Seq2SeqModel(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, embed_size=256, hidden_size=512, dropout=0.4):
        super(Seq2SeqModel, self).__init__()
        self.encoder_embedding = nn.Embedding(input_vocab_size, embed_size, padding_idx=0)
        self.encoder_lstm = nn.LSTM(embed_size, hidden_size, batch_first=True, bidirectional=True)
        self.decoder_embedding = nn.Embedding(output_vocab_size, embed_size, padding_idx=0)
        self.decoder_lstm = nn.LSTM(embed_size, hidden_size * 2, batch_first=True, dropout=dropout)
        self.output_layer = nn.Linear(hidden_size * 2, output_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, encoder_input, encoder_lengths, decoder_input):
        encoder_embedded = self.encoder_embedding(encoder_input)
        packed_input = pack_padded_sequence(encoder_embedded, encoder_lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_outputs, (h_n, c_n) = self.encoder_lstm(packed_input)
        encoder_outputs, _ = pad_packed_sequence(packed_outputs, batch_first=True)
        h_n = torch.cat((h_n[0], h_n[1]), dim=-1)
        c_n = torch.cat((c_n[0], c_n[1]), dim=-1)
        encoder_states = (h_n.unsqueeze(0), c_n.unsqueeze(0))
        decoder_embedded = self.decoder_embedding(decoder_input)
        decoder_outputs, _ = self.decoder_lstm(decoder_embedded, encoder_states)
        decoder_outputs = self.dropout(decoder_outputs)
        decoder_outputs = self.output_layer(decoder_outputs)
        return decoder_outputs

In [ ]:
input_vocab_size = input_tokenizer.vocab_size
output_vocab_size = output_tokenizer.vocab_size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Seq2SeqModel(input_vocab_size, output_vocab_size).to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
print(model)

Seq2SeqModel(
  (encoder_embedding): Embedding(30522, 256, padding_idx=0)
  (encoder_lstm): LSTM(256, 512, batch_first=True, bidirectional=True)
  (decoder_embedding): Embedding(30522, 256, padding_idx=0)
  (decoder_lstm): LSTM(256, 1024, batch_first=True, dropout=0.4)
  (output_layer): Linear(in_features=1024, out_features=30522, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)


In [ ]:
print("Starting training...")  # Debug print

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs} started")  # Debug print
    model.train()

    total_loss = 0
    correct = 0  # Track correct predictions
    total = 0  # Track total predictions

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        lengths = attention_mask.sum(dim=1)  # Get actual sequence lengths

        optimizer.zero_grad()
        outputs = model(input_ids, lengths, labels)  # Forward pass

        loss = loss_fn(outputs.view(-1, output_vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Compute Accuracy
        predictions = outputs.argmax(dim=-1)  # Get predicted token indices
        mask = labels != 0  # Ignore padding tokens
        correct += (predictions[mask] == labels[mask]).sum().item()
        total += mask.sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = (correct / total) * 100 if total > 0 else 0  # Avoid division by zero

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete!")  # Debug print

Starting training...
Epoch 1/5 started
Epoch 1/5, Loss: 0.3787, Accuracy: 96.46%
Epoch 2/5 started
Epoch 2/5, Loss: 0.0526, Accuracy: 99.50%
Epoch 3/5 started
Epoch 3/5, Loss: 0.0250, Accuracy: 99.76%
Epoch 4/5 started
Epoch 4/5, Loss: 0.0177, Accuracy: 99.86%
Epoch 5/5 started
Epoch 5/5, Loss: 0.0123, Accuracy: 99.91%
Training complete!


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)  # Move to GPU/CPU

In [ ]:
# Check if BOS token exists, otherwise use EOS token
bos_token_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id

# If still None, fallback to a default token (e.g., padding token or index 0)
if bos_token_id is None:
    bos_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

# Now create the decoder input tensor
decoder_input = torch.tensor([[bos_token_id]], dtype=torch.long)

# Ensure all tensors are on CPU
encoder_input = encoder_input.to("cpu")
encoder_lengths = encoder_lengths.to("cpu")
decoder_input = decoder_input.to("cpu")

# Run inference
with torch.no_grad():
    output = model(encoder_input, encoder_lengths, decoder_input)

# Get predicted token IDs (modify based on your Seq2Seq model)
predicted_ids = output.argmax(dim=-1)  # Ensure this aligns with your model’s output

# Convert to text
response = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
print("Generated Response:", response)

Generated Response: 


In [ ]:

model.to("cpu")
torch.save(model, "model_cpu_full.pth")

print("Model saved in CPU mode!")


Model saved in CPU mode!


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tensorflow.keras.models import load_model


# Load DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Function to generate chatbot response
def generate_response(user_input, max_length=150):
    # Encode user input and add end-of-sentence token
    input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")

    # Generate response
    chat_history_ids = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the generated response
    response_text = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response_text

# Interactive chat
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break

    response = generate_response(user_input)
    print(f"Chatbot: {response}")

You: hi
Chatbot: Hey! :D
You: quit
Chatbot: Goodbye!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model name
model_name = "microsoft/DialoGPT-medium"

# Download tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save tokenizer and model
tokenizer.save_pretrained("./dialo_model/")
model.save_pretrained("./dialo_model/")

# Compress the model for easy transfer
import shutil
shutil.make_archive("DialoGPT_model", 'zip', "./dialo_model/")

print("DialoGPT model saved as DialoGPT_model.zip")

DialoGPT model saved as DialoGPT_model.zip
